In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFDirectoryLoader
import chromadb
from chromadb.utils import embedding_functions
import uuid

In [2]:
def load_dir(dir):
    loader = PyPDFDirectoryLoader(dir)
    documents = loader.load()
    return documents

In [22]:
def chunking(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=20, separators=["\n\n", "\n", " ", ""])
    chunks = text_splitter.split_documents(documents)
    ids = [str(uuid.uuid1()) for _ in range(len(chunks))] 
    texts = [chunk.page_content for chunk in chunks]
    metadata = [chunk.metadata for chunk in chunks] 
    return ids, texts, metadata

In [3]:
def embed_chunks(ids, chunks, metadata) -> None:
    embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")
    client = chromadb.PersistentClient(path="./vector_db/")
    collection = client.get_or_create_collection(
        name='policies',
        embedding_function=embedding_function
    )
    collection.add(
        documents=chunks,
        metadatas=metadata,
        ids=ids
    )

In [8]:
if __name__ == '__main__':
    docs = load_dir('RAG docs/')
    ids, chunks, metadata = chunking(docs)
    embed_chunks(ids, chunks, metadata)

In [19]:
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")
client = chromadb.PersistentClient(path="./vector_db/")
collection = client.get_or_create_collection(
    name='policies',
    embedding_function=embedding_function
)
    
query = 'All policies and guidelines related to the customer'

results = collection.query(
    query_texts=[query],
    n_results=5,
    include=["documents", "metadatas"]
)

In [20]:
print(''.join([rules for rules in results['documents'][0]]))


• Not enter the customer’s residence against his/her wishes or when they are told the customer is not at home;  
• Not restrict the customer’s movement or restrain him/her from entering or leaving the house/room;  
• Not remain in the  customer’s house if he/she were to leave for any reason including to collect money from a bank/elsewhere;  
• Respect the customer’s privacy – do not embarrass the customer in the presence of his/her neighbours;• Not enter the customer’s residence against his/her wishes or when they are told the customer is not at home;  
• Not restrict the customer’s movement or restrain him/her from entering or leaving the house/room;  
• Not remain in the  customer’s house if he/she were to leave for any reason including to collect money from a bank/elsewhere;  
• Respect the customer’s privacy – do not embarrass the customer in the presence of his/her neighbours;every interaction with customers and the Customers must be informed that the call is being recorded.  
4. 